In [1]:
import os
import pyproj
import geopandas as gpd
import numpy as np
import rasterio
from rasterio import features
from shapely.geometry import box, shape
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import warnings

# 忽略栅格化函数的弃用警告
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 设置 PROJ 数据目录
pyproj.datadir.set_data_dir("/home/yifan/anaconda3/envs/myenv/share/proj")
print("PROJ 数据目录:", pyproj.datadir.get_data_dir())

############################
# 1. 栅格化函数
############################
def rasterize_polygon(gdf, transform, width, height, fill_value=0, dtype='int16'):
    """
    将GeoDataFrame中的几何体栅格化为给定范围和分辨率的numpy数组。
    
    参数:
    - gdf: 要栅格化的GeoDataFrame
    - transform: rasterio.transform 对象
    - width, height: 输出栅格的宽和高
    - fill_value: 未覆盖区域的像素值
    - dtype: 输出栅格的数据类型
    
    返回:
    - raster: 栅格化后的numpy数组
    """
    shapes = [(geom, 1) for geom in gdf.geometry if geom is not None and not geom.is_empty]
    raster = features.rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=fill_value,
        dtype=dtype
    )
    return raster

############################
# 2. 清理 GeoDataFrame 函数
############################
def clean_geodataframe(gdf, fields_to_check=None, drop_based_on_geometry=True, fix_invalid=True):
    """
    清理 GeoDataFrame，删除几何体为 None 或空的行，并根据需要修复无效几何体。
    
    参数:
    - gdf: 输入的 GeoDataFrame
    - fields_to_check: 需要检查空值的字段列表。如果为 None，则不基于属性字段清理。
    - drop_based_on_geometry: 是否基于几何体清理（默认是 True）
    - fix_invalid: 是否尝试修复无效的几何体（默认是 True）
    
    返回:
    - cleaned_gdf: 清理后的 GeoDataFrame
    """
    initial_count = len(gdf)
    if drop_based_on_geometry:
        # 删除几何体为 None 或空的行
        gdf = gdf[gdf.geometry.notnull() & ~gdf.geometry.is_empty]
        removed = initial_count - len(gdf)
        print(f"删除几何体为 None 或空的行: {removed}")
        initial_count = len(gdf)
    
    # 如果指定了字段，删除这些字段中有空值的行
    if fields_to_check:
        gdf = gdf.dropna(subset=fields_to_check)
        removed = initial_count - len(gdf)
        print(f"删除属性字段空值的行: {removed}")
    else:
        print("未基于属性字段清理 GeoDataFrame。")
    
    # 尝试修复无效的几何体
    if fix_invalid:
        invalid = ~gdf.is_valid
        if invalid.any():
            print(f"尝试修复 {invalid.sum()} 个无效的几何体。")
            gdf.loc[invalid, 'geometry'] = gdf.loc[invalid, 'geometry'].buffer(0)
            # 检查修复后的几何体是否有效
            still_invalid = ~gdf.is_valid
            if still_invalid.any():
                print(f"警告: 仍有 {still_invalid.sum()} 个几何体无效，已删除。")
                gdf = gdf[gdf.is_valid]
            else:
                print("所有几何体已成功修复。")
    
    final_count = len(gdf)
    print(f"清理前总行数: {initial_count}, 清理后总行数: {final_count}")
    return gdf

############################
# 3. 裁剪 GeoDataFrame 函数
############################
def clip_geodataframe(gdf, clip_gdf):
    """
    根据裁剪边界裁剪 GeoDataFrame。
    
    参数:
    - gdf: 要裁剪的 GeoDataFrame
    - clip_gdf: 用于裁剪的 GeoDataFrame（通常是一个或多个几何体）
    
    返回:
    - clipped_gdf: 裁剪后的 GeoDataFrame
    """
    # 获取裁剪边界的联合几何体
    clip_boundary = clip_gdf.unary_union
    
    # 使用 geopandas 的 clip 函数进行裁剪
    clipped_gdf = gpd.clip(gdf, clip_boundary)
    
    return clipped_gdf

############################
# 4. 验证 GeoDataFrame 函数
############################
def verify_geodataframe(gdf, name):
    """
    验证 GeoDataFrame 的有效性，并打印相关信息。
    
    参数:
    - gdf: 要验证的 GeoDataFrame
    - name: GeoDataFrame 的名称，用于打印信息
    """
    print(f"\n验证 {name} GeoDataFrame:")
    print(f"总行数: {len(gdf)}")
    print(f"CRS: {gdf.crs}")
    
    if gdf.empty:
        print(f"警告: {name} GeoDataFrame 是空的。")
    else:
        # 检查几何体有效性
        invalid_geometries = ~gdf.is_valid
        if invalid_geometries.any():
            print(f"警告: {name} GeoDataFrame 中存在无效的几何体。")
            print(gdf[invalid_geometries])
        else:
            print(f"{name} GeoDataFrame 中所有几何体均有效。")

############################
# 5. 获取检测结果栅格参数
############################
def get_detection_raster_params(detection_gdf, pixel_size=0.0001, raster_width=256, raster_height=256):
    """
    根据检测结果的 GeoDataFrame，获取栅格化所需的 transform、width、height 等参数。
    
    参数:
    - detection_gdf: 检测结果的 GeoDataFrame
    - pixel_size: 像素分辨率（可选）
    - raster_width: 输出栅格的宽度（默认256）
    - raster_height: 输出栅格的高度（默认256）
    
    返回:
    - transform: rasterio.transform 对象
    - width: 栅格宽度
    - height: 栅格高度
    """
    # 获取检测结果的边界框
    bounds = detection_gdf.total_bounds  # (minx, miny, maxx, maxy)
    minx, miny, maxx, maxy = bounds
    
    # 定义栅格的仿射变换
    transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, raster_width, raster_height)
    
    print(f"检测结果栅格参数: transform={transform}, width={raster_width}, height={raster_height}")
    return transform, raster_width, raster_height

############################
# 6. 删除非森林区域像素
############################
def mask_non_forest_pixels(prediction_raster, forest_raster):
    """
    将不在森林范围（forest_raster=0）内的像素置为0。
    
    参数:
    - prediction_raster: 预测结果栅格（numpy数组）
    - forest_raster: 森林掩膜栅格（numpy数组）
    
    返回:
    - masked_result: 处理后的预测栅格
    """
    masked_result = prediction_raster.copy()
    masked_result[forest_raster == 0] = 0
    return masked_result

############################
# 7. 计算评估指标（手动方法）
############################
def calculate_metrics_between_rasters(pred_raster, true_raster):
    """
    计算两个二值栅格（预测与真实标注）之间的 TP, FP, FN, Precision, Recall, F1-score, IoU。
    
    参数:
    - pred_raster: 预测结果栅格（numpy数组，1表示正类，0表示负类）
    - true_raster: 真实标注栅格（numpy数组，1表示正类，0表示负类）
    
    返回:
    - TP: 真正例的数量
    - FP: 假正例的数量
    - FN: 假负例的数量
    - precision: 精确率
    - recall: 召回率
    - f1: F1 分数
    - iou: 交并比 (Intersection over Union)
    """
    assert pred_raster.shape == true_raster.shape, "两个栅格的尺寸必须相同"

    # 计算 TP, FP, FN
    TP = np.sum((pred_raster == 1) & (true_raster == 1))
    FP = np.sum((pred_raster == 1) & (true_raster == 0))
    FN = np.sum((pred_raster == 0) & (true_raster == 1))

    # 计算 Precision, Recall, F1-score, IoU
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0.0

    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0.0

    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0

    if TP + FP + FN > 0:
        iou = TP / (TP + FP + FN)
    else:
        iou = 0.0

    return TP, FP, FN, precision, recall, f1, iou

############################
# 8. 计算评估指标（使用 sklearn）
############################
def calculate_metrics_with_sklearn(pred_raster, true_raster):
    """
    使用 sklearn 库计算两个二值栅格（预测与真实标注）之间的 Precision, Recall, F1-score 和 IoU。
    
    参数:
    - pred_raster: 预测结果栅格（numpy数组，1表示正类，0表示负类）
    - true_raster: 真实标注栅格（numpy数组，1表示正类，0表示负类）
    
    返回:
    - precision: 精确率
    - recall: 召回率
    - f1: F1 分数
    - iou: 交并比 (Intersection over Union)
    """
    # 展平栅格以进行像素级别的对比
    pred_flat = pred_raster.flatten()
    true_flat = true_raster.flatten()

    # 只考虑 pred_raster 和 true_raster 中非0的部分，避免计算过程中包含无效区域
    valid_mask = (pred_flat != 0) | (true_flat != 0)

    y_true = true_flat[valid_mask]
    y_pred = pred_flat[valid_mask]

    # 二值化预测结果（确保为0或1）
    y_pred = (y_pred > 0).astype('int16')
    y_true = (y_true > 0).astype('int16')

    # 检查 y_true 和 y_pred 的分布，避免全为一类的情况
    if y_true.sum() == 0 or y_pred.sum() == 0:
        print("警告：y_true 或 y_pred 中没有正样本或负样本，可能导致召回率或精确率异常。")
        return None, None, None, None

    # 计算 Precision, Recall, F1-score
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # 计算 IoU
    iou = jaccard_score(y_true, y_pred, zero_division=0)

    return precision, recall, f1, iou

############################
# 9. 将二值栅格转为矢量并保存
############################
def save_raster_as_shapefile(raster, transform, crs, output_path="masked_detection_raster.shp"):
    """
    将二值栅格转换为多边形并保存为 shapefile。
    
    参数:
    - raster: numpy 数组，二值栅格（0或1）
    - transform: 对应的仿射变换
    - crs: 输出矢量文件所用的坐标参考系
    - output_path: 保存的 shapefile 路径（默认保存到当前目录）
    """
    # 将像素值转换为矢量多边形
    shapes_generator = features.shapes(raster, transform=transform)
    
    polygons = []
    for geom_dict, val in shapes_generator:
        # 只将值为 1 的区域转换为多边形
        if val == 1:
            polygons.append(shape(geom_dict))

    # 构建 GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)

    # 如果你想给输出添加其他属性列，可以在此添加
    # 例如: gdf["some_attribute"] = 1

    # 保存为 shapefile
    gdf.to_file(output_path)
    print(f"成功将二值栅格保存为 shapefile: {output_path}")

############################
# 10. 主流程示例
############################
def main_evaluation(forest_mask_path, annotation_path, detection_result_path, pixel_size=0.0001):
    """
    主流程:
    1) 读取和栅格化 detection_result, forest_mask, annotation 到相同范围与大小
    2) 删除具有空值的多边形
    3) 验证 forest_mask 和 annotation 是否正确读取
    4) 裁剪 forest_mask 和 annotation 以匹配 detection_result 的边界
    5) 将 detection_result 中不属于森林区域的像素置为0
    6) 与 annotation 比较并计算评估指标（手动和 sklearn 方法）
    
    参数:
    - forest_mask_path: 森林掩膜的矢量文件路径
    - annotation_path: 手动标注的矢量文件路径
    - detection_result_path: 检测结果的矢量文件路径
    - pixel_size: 像素分辨率（可选）
    """
    
    # 检查文件是否存在
    for path in [forest_mask_path, annotation_path, detection_result_path]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"文件未找到: {path}")
    
    # 读取检测结果矢量文件并清理空值多边形（仅基于几何体）
    detection_gdf = gpd.read_file(detection_result_path)
    detection_gdf = clean_geodataframe(detection_gdf, fields_to_check=None, drop_based_on_geometry=True)
    
    # 验证 detection_gdf
    verify_geodataframe(detection_gdf, "Detection Result")
    
    # 检查检测结果是否为空
    if detection_gdf.empty:
        raise ValueError("检测结果文件中没有有效的多边形。")
    
    # 确保所有 GeoDataFrame 使用相同的 CRS
    # 假设使用检测结果的 CRS 作为统一的 CRS
    unified_crs = detection_gdf.crs
    if unified_crs is None:
        raise ValueError("检测结果的 CRS 未定义，请确保所有输入文件具有定义的 CRS。")
    
    # 读取森林掩膜和标注文件，并清理空值多边形（仅基于几何体）
    forest_gdf = gpd.read_file(forest_mask_path)
    forest_gdf = clean_geodataframe(forest_gdf, fields_to_check=None, drop_based_on_geometry=True)
    forest_gdf = forest_gdf.to_crs(unified_crs)  # 转换 CRS
    
    annotation_gdf = gpd.read_file(annotation_path)
    annotation_gdf = clean_geodataframe(annotation_gdf, fields_to_check=None, drop_based_on_geometry=True)
    annotation_gdf = annotation_gdf.to_crs(unified_crs)  # 转换 CRS
    
    # 验证 forest_gdf 和 annotation_gdf
    verify_geodataframe(forest_gdf, "Forest Mask")
    verify_geodataframe(annotation_gdf, "Annotation")
    
    # 裁剪 forest_gdf 和 annotation_gdf 以匹配 detection_gdf 的边界
    forest_gdf_clipped = clip_geodataframe(forest_gdf, detection_gdf)
    annotation_gdf_clipped = clip_geodataframe(annotation_gdf, detection_gdf)
    
    # 验证裁剪后的 GeoDataFrame
    verify_geodataframe(forest_gdf_clipped, "Clipped Forest Mask")
    verify_geodataframe(annotation_gdf_clipped, "Clipped Annotation")
    
    # 检查裁剪后的 GeoDataFrame 是否为空
    if forest_gdf_clipped.empty:
        raise ValueError("裁剪后的森林掩膜文件中没有有效的多边形。")
    if annotation_gdf_clipped.empty:
        raise ValueError("裁剪后的标注文件中没有有效的多边形。")
    
    # 获取栅格范围和大小
    transform, width, height = get_detection_raster_params(
        detection_gdf,
        pixel_size=pixel_size,
        raster_width=256,
        raster_height=256
    )
    
    # 栅格化：森林掩膜
    forest_raster = rasterize_polygon(forest_gdf_clipped, transform, width, height)
    forest_raster = (forest_raster > 0).astype('int16')  # 二值化
    
    # 栅格化：标注
    annotation_raster = rasterize_polygon(annotation_gdf_clipped, transform, width, height)
    annotation_raster = (annotation_raster > 0).astype('int16')  # 二值化
    
    # 栅格化：检测结果
    detection_raster = rasterize_polygon(detection_gdf, transform, width, height)
    detection_raster = (detection_raster > 0).astype('int16')  # 二值化
    
    # 删除不属于森林的像素
    masked_detection_raster = mask_non_forest_pixels(detection_raster, forest_raster)
    
    # 计算评估指标（手动方法）
    TP, FP, FN, precision_manual, recall_manual, f1_manual, iou_manual = calculate_metrics_between_rasters(
        masked_detection_raster,
        annotation_raster
    )
    
    print("\n评估结果（手动计算）:")
    print(f"TP: {TP}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")
    print(f"Precision: {precision_manual:.4f}")
    print(f"Recall: {recall_manual:.4f}")
    print(f"F1-Score: {f1_manual:.4f}")
    print(f"IoU: {iou_manual:.4f}")
    
    # 计算评估指标（使用 sklearn）
    precision_sklearn, recall_sklearn, f1_sklearn, iou_sklearn = calculate_metrics_with_sklearn(
        masked_detection_raster,
        annotation_raster
    )
    
    if precision_sklearn is not None:
        print("\n评估结果（使用 sklearn）:")
        print(f"Precision: {precision_sklearn:.4f}")
        print(f"Recall: {recall_sklearn:.4f}")
        print(f"F1-Score: {f1_sklearn:.4f}")
        print(f"IoU: {iou_sklearn:.4f}")
    
    # ================
    # 保存输出为 shapefile
    # ================
    save_raster_as_shapefile(
        masked_detection_raster,
        transform,
        crs=unified_crs,
        output_path="masked_detection_raster.shp"  # 保存到当前目录
    )

# 如果需要直接测试，可取消注释并修改路径
if __name__ == "__main__":
    try:
        main_evaluation(
            "Forest_Mask_2021.shp",
            "622_975_2022.shp",
            "anomaly_difference_20220919.shp"
        )
    except FileNotFoundError as e:
        print(f"文件未找到错误: {e}")
    except ValueError as e:
        print(f"值错误: {e}")
    except Exception as e:
        print(f"发生未预料的错误: {e}")

PROJ 数据目录: /home/yifan/anaconda3/envs/myenv/share/proj
删除几何体为 None 或空的行: 0
未基于属性字段清理 GeoDataFrame。
清理前总行数: 347, 清理后总行数: 347

验证 Detection Result GeoDataFrame:
总行数: 347
CRS: EPSG:4326
Detection Result GeoDataFrame 中所有几何体均有效。
删除几何体为 None 或空的行: 0
未基于属性字段清理 GeoDataFrame。
清理前总行数: 676, 清理后总行数: 676
删除几何体为 None 或空的行: 3
未基于属性字段清理 GeoDataFrame。
尝试修复 4 个无效的几何体。
所有几何体已成功修复。
清理前总行数: 349, 清理后总行数: 349

验证 Forest Mask GeoDataFrame:
总行数: 676
CRS: EPSG:4326
Forest Mask GeoDataFrame 中所有几何体均有效。

验证 Annotation GeoDataFrame:
总行数: 349
CRS: EPSG:4326
Annotation GeoDataFrame 中所有几何体均有效。

验证 Clipped Forest Mask GeoDataFrame:
总行数: 136
CRS: EPSG:4326
Clipped Forest Mask GeoDataFrame 中所有几何体均有效。

验证 Clipped Annotation GeoDataFrame:
总行数: 241
CRS: EPSG:4326
Clipped Annotation GeoDataFrame 中所有几何体均有效。
检测结果栅格参数: transform=| 0.00, 0.00,-70.65|
| 0.00,-0.00,-8.41|
| 0.00, 0.00, 1.00|, width=256, height=256

评估结果（手动计算）:
TP: 898
FP: 186
FN: 410
Precision: 0.8284
Recall: 0.6865
F1-Score: 0.7508
IoU: 0.6011

评估结果（使用 sklearn）:
